# ASLDRO Example

This is an example jupyter notebook that can run ASLDRO in your browser.

In [ ]:
from asldro.examples import run_full_pipeline
from asldro.validators.user_parameter_input import get_example_input_params
from pprint import pprint as pp
import numpy as np
from numpy.random import default_rng
from tempfile import TemporaryDirectory
import json
import os
from IPython.display import FileLink, HTML


First, let's import the default user input parameters as a python dictionary.

In [ ]:
input_params = get_example_input_params() # import the default user parameters
pp(input_params) #view them (nicely)

Now, let's edit these default parameters so that we have something that mimics a "real" acquisition:
* Increase the number of acquisitions
* Add random motion
* Change the number of slices

See https://asldro.readthedocs.io/en/stable/usage/series-asl.html for more information on the ASL series parameters.

In [ ]:
asl_params = input_params["image_series"][0]["series_parameters"] # makes typing easier!
# First let's define the m0, control and label order and number of scans
number_control_label_pairs = 30
number_m0 = 1
number_volumes = number_control_label_pairs*2 + number_m0
# build the asl_context string
asl_params["asl_context"] = "m0scan"*number_m0 + " control label"*number_control_label_pairs
asl_params["echo_time"] = [0.01] * number_volumes # 10ms TE for every volume
asl_params["repetition_time"] = [10.0] + [5.0]*number_control_label_pairs*2 # 10ms for the M0, 5ms for each control or label acquisition.

# Now let's add some random motion
rng = default_rng(seed=90210)
# rotation with mean 0 and sd 0.1°
asl_params["rot_x"] = list(rng.normal(0, 0.1, number_volumes))
asl_params["rot_y"] = list(rng.normal(0, 0.1, number_volumes))
asl_params["rot_z"] = list(rng.normal(0, 0.1, number_volumes))
# translation with mean 0 and sd 0.1mm
asl_params["transl_x"] = list(rng.normal(0, 0.1, number_volumes))
asl_params["transl_y"] = list(rng.normal(0, 0.1, number_volumes))
asl_params["transl_z"] = list(rng.normal(0, 0.1, number_volumes))

# The input ground truth is at 1x1x1mm resolution and has dimensions [197, 233, 189]
# Let's adjust the output resolution so we have something close to 4x4x4mm voxels
asl_params["acq_matrix"] = np.ceil([197/4, 233/4, 189/4]).astype(np.int32).tolist()
#also change the ground truth image series to have the same acquisition resolution
input_params["image_series"][1]["series_parameters"]["acq_matrix"] = asl_params["acq_matrix"]

Let's run ASLDRO with these parameters:

In [ ]:
output_directory = os.path.join(os.getcwd(), "working")
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
archive_fn = os.path.join(output_directory, "dro_out.zip")
#run the full DRO pipeline, saving the results to archive_fn
run_full_pipeline(input_params, archive_fn)

#output the parameters to a json file inside the working directory
parameters_filename = os.path.join(output_directory, "dro_parameters.json")
with open(parameters_filename, "w") as json_file:
    json.dump(input_params, json_file, indent=4)

# display links to download the output and parameter files
display(HTML('''
    Results ready to download:
    <a href="./working/dro_out.zip" download="dro_out.zip">Download DRO Output</a>
    <br>
    Parameters ready to download:
    <a href="./working/dro_parameters.json" download="dro_parameters.json">Download DRO Parameters</a>
    '''))

